In [9]:
# Importing librairies

from jiwer import wer, cer, mer, wip, wil
import re
import csv
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import string
import operator
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2')

In [2]:
def get_dataset(data):
    """Gets and returns filename and sentences from a dataset.
    
    Parameters
    ----------
        data : str
        Path to csv file.
        
    Returns
    -------
        dict_values : dict
        Audio filename as key and its written transcription as value.
    """
    dict_values = {}
    
    with open(data, newline='') as f:
        reader = csv.reader(f, delimiter=' ', quotechar='|')
        for row in reader:
            row = ' '.join(row).split('\t')
            dict_values[row[0]] = row[1].lower()
            
        f.close()
    
    return dict_values

In [3]:
train = get_dataset("../ground_truth/truth_train.csv")
test = get_dataset("../ground_truth/truth_train.csv")
val = get_dataset("../ground_truth/truth_train.csv")
performed = get_dataset("output_performed_wav2vec2/w2v_performed_train.csv")

In [6]:
def get_errors(data_t,data_p):
    
    for key in data_t.keys():
        if data_t[key].split() != data_p[key].split():
            
            if data_t[key].replace(" ", "") != data_p[key].replace(" ", ""):
                print(data_t[key],data_p[key])
                
                w1_cor = ""
                w2_err = ""
                errors_count = 0
                errors_in_position = 0
                
                for count, (w1,w2) in enumerate(zip(data_t[key].split(),data_p[key].split())):
                    if w1 != w2:
                        errors_count += 1
                        w1_cor = w1
                        w2_err = w2

In [7]:
get_errors(train,performed)

в поході брали участь князі андрій і олександр так званий невський у поході брали участь князі андрій і олександер так званий невський
ходь ближчій махнув до нього маленький старчик у довгій непідперезаній сорочці хоть ближчіймахнув до нього маленький старчик у довгій непідперезаній сорочці
лізь речу заквапив товариша людота бо за півгоні внизу чулося стримане крякання лізь речу заквапив товариша людота бо за півгоні не з у чулося стримане крякання
гано ліпше відає він ходив у тоті краї гано ліпше відає він ходив у тут і краї
але вкривши чорне волосся смушевим ковпаком він таки пішов сідлати коня тихо ворушачи губами але вкривши чорне волось смушевим ковпаком він таки пішов сідлати коня тихо ворушачи губами
сватоплук знову порадився зі своїми мунжами й відповів сватоплук знову порадився зі своїми муншами й відповів
зодкуду знаєш порусинському зокуду знаєш по русинському
то речеш ні братті їй сестер устряв у розмову борислав то речеш ні братті їй сестерустряв розмову борислав
дай княже 

In [24]:
def create_dataset(data_train,data_test,data_val):
    
    sentences_train = [sentence for sentence in data_train.values()]
    sentences_test = [sentence for sentence in data_test.values()]
    sentences_val = [sentence for sentence in data_val.values()]
    
    sentence_text = sentences_train + sentences_test + sentences_val
    sentence = []
    
    for item in sentence_text:
        sentence.extend(item.split())
    
    with open("vocabulary.csv", 'w') as f:
        writer = csv.writer(f, delimiter='\n')
        writer.writerow(sentence)
        f.close()
        
    with open("vocabulary_set.csv", 'w') as f:
        writer = csv.writer(f, delimiter='\n')
        writer.writerow(set(sentence))
        f.close()
    
    return sentence

data_text = create_dataset(train,test,val)

In [7]:
def get_data_for_emb(data):
    
    list_values = []
    list_values_error = []
    
    with open(data, newline='') as f:
        reader = csv.reader(f, delimiter=' ')
        for row in reader:
            list_values.append(row[0])
            list_values_error.append(row[1])
            
    return list_values, list_values_error

truth_emb, performed_emb = get_data_for_emb("../datasets_machine_translation/train_truth_performed.csv")
truth_emb_error, performed_emb_error = get_data_for_emb("../datasets_machine_translation/train_truth_performed_errors_only.csv")

In [ ]:
#word embeddings to match synonyms
#run the similarity between truth VS performed
# https://www.sbert.net/docs/usage/semantic_textual_similarity.html

def get_embeddings(data_truth,data_error):
    """Gets embeddings from dataset.
    
    Parameters
    ----------
    data_truth : list
        A list of sentences from ground truth.
        
    data_error : list
        A list of performed sentences not matching ground truth.
        
    Returns
    -------
    output_emb : list
        List of sentences and their embedding.
    """
    
    output_emb = []
    
    embeddings_t = model.encode(data_truth, convert_to_tensor=True)
    embeddings_e = model.encode(data_error, convert_to_tensor=True)
    
    #Compute cosine-similarits
    cosine_scores = util.cos_sim(embeddings_t, embeddings_e)
    
    #Output the pairs with their score
    for i in range(len(embeddings_t)):
        print("{} \t\t {} \t\t Score: {:.4f}".format(embeddings_t[i], embeddings_e[i], cosine_scores[i][i]))
    
    #Find the pairs with the highest cosine similarity scores
    pairs = []
    for i in range(len(cosine_scores)-1):
        for j in range(i+1, len(cosine_scores)):
            pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})
    #Sort scores in decreasing order
    pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)
    for pair in pairs[0:10]:
        i, j = pair['index']
        print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], pair['score']))
    
    #return output_emb

get_embeddings(truth_emb_error,performed_emb_error)